# Use Keras/LSTM Model to Generate Fake Amazon Reviews

## Load Libraries

In [1]:
import numpy as np
import pandas as pd
import json
import keras
from keras import layers
import random
import sys

## Read in Product Data

We will use the description/title of the product as a seed for the review generatin so we don't write a review about the wrong product!

In [27]:
# read in products
products = pd.read_json('./data/meta_Office_Products.json',lines=True)

In [28]:
products.head(10)

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Office Products, Office & School Supplies, Ed...","class=""a-keyvalue prodDetTable"" role=""present...",[Sequential Spelling is based on the classic O...,,Sequential Spelling Level 1 Bundle with Studen...,[],[],,STL Distributors,[],"[>#439,654 in Office Products (See top 100), >...","[1935943065, 1935943073, B00IJH9Q4M, 002115021...",Office Products,"class=""a-bordered a-horizontal-stripes a-spa...","August 15, 2014",$32.90,0012624861,NaN
1,"[Office Products, Office &amp; School Supplies...",,"[Unusual book, , ]",,"Mathematics, Applications and Concepts, Course...",[],[],,bailey,[],"3,839,628 in Books (",[],Books,,,$8.62,0078652669,NaN
2,[],"class=""a-keyvalue prodDetTable"" role=""present...",[Pearson MyHistoryLab Online Access Code for A...,,Pearson MyHistoryLab Online Access Code for Am...,[],[],,Pearson MyHistoryLab,[Pearson MyHistoryLab Online Access Code for A...,"[>#1,925,354 in Office Products (See top 100)]",[],Office Products,,"June 21, 2012",$0.99,0136039847,NaN
3,"[Office Products, Office & School Supplies, Ed...","class=""a-keyvalue prodDetTable"" role=""present...",[Corduroy the bear goes to the launderette wit...,,A Pocket for Corduroy,"[0140501738, 0448421917, 0670063428, 042528875...",[],,Ingram Book & Distributor,[9780140503524],"[>#422,894 in Office Products (See top 100), >...",[0140501738],Office Products,,"September 14, 2006",$0.95,0140503528,NaN
4,"[Office Products, Office & School Supplies, Ed...",,"[<div class=""aplus""> <div class=""leftImage"" st...",,Social Entrepreneurship: What Everyone Needs t...,"[0195334760, 1613630328, 1422104060, 158648956...",[],,Visit Amazon's David Bornstein Page,[],"110,732 in Books (","[0195334760, 1586489569, 1613630328, 142210406...",Books,,,,0195396332,NaN
5,"[Office Products, Office & School Supplies, Bo...","class=""a-keyvalue prodDetTable"" role=""present...",[A good helper to help you record your reading...,,EKLOEN Mixed Designs of Antiqued Bronze Colour...,"[B00BLY6POE, B00G8WV5U8, 0307591662, B019XJZHQ...",[https://images-na.ssl-images-amazon.com/image...,,EKLOEN,"[It is not only the bookmarks, but also art, I...","[>#43,748 in Office Products (See top 100), >#...",[],Office Products,"class=""a-bordered a-horizontal-stripes a-spa...","November 8, 2015",$17.50,0245109919,NaN
6,"[Office Products, Office &amp; School Supplies...",,"[, ]",,Rachael Ray's Refrigerator Notes: 3 Magnetic P...,[],[],,Visit Amazon's Rachael Ray Page,[],"5,283,745 in Books (",[],Books,,,,0307236315,NaN
7,"[Office Products, Office &amp; School Supplies...",,"[Paulo Coelho, regarded by millions as an alch...",,Paulo Coelho Moments 2012 Day Planner,[0525435077],[],,Visit Amazon's Paulo Coelho Page,[],"3,872,855 in Books (",[1101972645],Books,,,,030794655X,NaN
8,"[Office Products, Office &amp; School Supplies...",,"[, ]",,Tri-Fold Organizer Black XXL Book and Bible Cover,"[031043758X, 1934770132, B00W4E1TKU, 031080917...",[],,Visit Amazon's Zondervan Page,[],"787,995 in Books (","[0310809177, B003JAH9MU, B0007UQKO8, 031082370...",Books,,,,0310432065,NaN
9,"[Office Products, Office &amp; School Supplies...",,"[, ]",,Adventure Bible Cover Pink Medium,"[0310802636, 0310727510, 0310727472, 031072744...",[],,Visit Amazon's Zondervan Page,[],"11,622 in Books (","[0310727472, 0310727421, B0793FF3N5, 031080263...",Books,,,$13.69,0310520347,NaN


In [29]:
products.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'image',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'details'],
      dtype='object')

### Clean up Product Data
We don't need a lot of this data for my purpose, and since this is a large data set, drop it

In [30]:
# We don't need a lot of this data for my purpose, and since this is a large data set, drop it
products.drop(['category','tech1','fit','also_buy','image','tech2',
               'feature','rank','also_view','similar_item','date','details'],
             axis=1, inplace=True)

In [31]:
products.head(10)

,description,title,brand,main_cat,price,asin
0,[Sequential Spelling is based on the classic O...,Sequential Spelling Level 1 Bundle with Studen...,STL Distributors,Office Products,$32.90,0012624861
1,"[Unusual book, , ]","Mathematics, Applications and Concepts, Course...",bailey,Books,$8.62,0078652669
2,[Pearson MyHistoryLab Online Access Code for A...,Pearson MyHistoryLab Online Access Code for Am...,Pearson MyHistoryLab,Office Products,$0.99,0136039847
3,[Corduroy the bear goes to the launderette wit...,A Pocket for Corduroy,Ingram Book & Distributor,Office Products,$0.95,0140503528
4,"[<div class=""aplus""> <div class=""leftImage"" st...",Social Entrepreneurship: What Everyone Needs t...,Visit Amazon's David Bornstein Page,Books,,0195396332
5,[A good helper to help you record your reading...,EKLOEN Mixed Designs of Antiqued Bronze Colour...,EKLOEN,Office Products,$17.50,0245109919
6,"[, ]",Rachael Ray's Refrigerator Notes: 3 Magnetic P...,Visit Amazon's Rachael Ray Page,Books,,0307236315
7,"[Paulo Coelho, regarded by millions as an alch...",Paulo Coelho Moments 2012 Day Planner,Visit Amazon's Paulo Coelho Page,Books,,030794655X
8,"[, ]",Tri-Fold Organizer Black XXL Book and Bible Cover,Visit Amazon's Zondervan Page,Books,,0310432065
9,"[, ]",Adventure Bible Cover Pink Medium,Visit Amazon's Zondervan Page,Books,$13.69,0310520347


In [32]:
# Product description is a list, let's grab the first entry in the list and call it done.
products['description'] = products['description'].apply(pd.Series)[0]

In [23]:
# save it for future
products.to_csv('./data/products.clean.csv')

### Read Review Data
The previous reviews will be used to build our corpus.

In [2]:
# read in reviews
reviews = pd.read_json('./data/Office_Products_5.json',lines=True)

In [3]:
reviews.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"11 7, 2017",A2NIJTYWADLK57,0140503528,{'Format:': ' Board book'},cotton clay,kids like story BUT while i really wanted a bo...,"good story, small size book though",1510012800,NaN,NaN
1,4,True,"03 7, 2017",A2827D8EEURMP4,0140503528,{'Format:': ' Hardcover'},emankcin,Bought this used and it came in great conditio...,Good,1488844800,NaN,NaN
2,5,True,"06 25, 2016",APB6087F4J09J,0140503528,{'Format:': ' Board book'},Starbucks Fan,Every story and book about Corduroy is Fantast...,Best Books for All Children,1466812800,NaN,NaN
3,5,True,"02 21, 2016",A2DHERRZIPFU7X,0140503528,{'Format:': ' Paperback'},Caitlyn Jacobson,I purchased this book for my first grade class...,Great for Math!,1456012800,NaN,NaN
4,5,False,"08 2, 2015",A2XCLJRGFANRC,0140503528,{'Format:': ' Hardcover'},E. Ervin,Having spent numerous years in an elementary s...,Love Corduroy,1438473600,NaN,NaN


### Build Corpus

In [4]:
# data is too big, get a random sample
reviews_sample = reviews.sample(frac=.05, replace=False, random_state=17)

In [5]:
corpus = reviews_sample['reviewText'].str.cat(sep='\n')

In [6]:
# full corpus length(all reviews): 193333114
# 3 & 4 corpus length            : 134511214
# 5% sample                      : 
print(f'Corpus length: {len(corpus)}')

Corpus length: 9756448


In [7]:
# extract sequences of 20 characters
maxlen = 25

# sample new sequence every 3 characters
step = 3

# hold extracted sequences
sentences = []

# hold targets
next_chars = []

In [8]:
# full sequences: 64444352
# 3 & 4         : 44837052
# 5%            :  2230245
for i in range(0, len(corpus) - maxlen, step):
    sentences.append(corpus[i: i + maxlen])
    next_chars.append(corpus[i + maxlen])
    
print('Number of sequences:', len(sentences))

Number of sequences: 3252141


In [9]:
# sorted list of unique characters in text
chars = sorted(list(set(corpus)))
print('Unique characters:', len(chars))

Unique characters: 97


In [10]:
# dictionary that maps unique characters to index in list {chars}
char_indices = dict((char, chars.index(char)) for char in chars)

In [11]:
# one hot encode characters into binary arrays

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [12]:
# Single-layer lstm model
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [13]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [14]:
def sample(preds, temparature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
for epoch in range(1, 60):
    print(f'epoch {epoch}')
    model.fit(x, y, batch_size=128, epochs=1)
    
    # open a file to write results to for certain epochs
    with  open(f"./results/epoch_{epoch}.txt","w") as outFile:
        # select text seed at random
        start_index = random.randint(0, len(corpus) - maxlen - 1)
        seed_text = corpus[start_index: start_index + maxlen]

        outFile.write(f'\n--- Generating with seed:"{seed_text}"\n')
        print(f'\n--- Generating with seed:"{seed_text}"\n')

        for temperature in [0.2, 0.5, 1.0, 1.2]:
            outFile.write(f'\n------ temperature: {temperature}\n')
            print(f'\n------ temperature: {temperature}\n')

            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))

                # one hote encode chracters generated so far
                for t, char in enumerate(seed_text):
                    sampled[0, t, char_indices[char]] = 1

                # predict and sample the next characther
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]

                seed_text += next_char
                seed_text = seed_text[1:]

                outFile.write(next_char)
    

epoch 1
25408/25408 [==============================] - 834s 33ms/step - loss: 1.7758

--- Generating with seed:"ome neat little mounting "


------ temperature: 0.2



<ipython-input-14-c17b164e6a93>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature



------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 2
25408/25408 [==============================] - 843s 33ms/step - loss: 5.1438

--- Generating with seed:"s a great mechanical penc"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 3
25408/25408 [==============================] - 887s 35ms/step - loss: 3.2440

--- Generating with seed:"rfect condition.  And tha"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 4
25408/25408 [==============================] - 797s 31ms/step - loss: 3.0800

--- Generating with seed:"ink seems thinner to me t"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 5
25408/25408 [==============================] - 812s 32ms/step - loss: 3.1008

--- Generating with seed:" experimentation.

E.M. V"


------ temperature: 0.2


------ temperature: 0.5

epoch 35
25408/25408 [==============================] - 721s 28ms/step - loss: 3.2479

--- Generating with seed:"is a very nice journal an"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 36
25408/25408 [==============================] - 717s 28ms/step - loss: 3.2467

--- Generating with seed:"ried generic, but Canon i"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 37
25408/25408 [==============================] - 715s 28ms/step - loss: 3.2508

--- Generating with seed:" until it snaps/pops. The"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 38
25408/25408 [==============================] - 714s 28ms/step - loss: 3.2600

--- Generating with seed:"l.

Bottom line - we're v"


------ temperature: 0.2


------ temperature: 0.5


------ temperature: 1.0


------ temperature: 1.2

epoch 39
25408/25408